In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/NLP_project/atten

/content/drive/My Drive/NLP_project/atten


In [3]:
ls

glove.840B.300d.txt  sarcasm_data.json  src/


In [4]:
import json
import re
import csv
import nltk
import math
# import gensim = 3.8.3
import string
import numpy as np
import pandas as pd
from nltk.util import ngrams
# from num2words import num2words
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
df = pd.read_json ('sarcasm_data.json')
df = df.T
def get_labels(text):
    if(text == True):
        return 1
    return 0

df['sarcasm'] = df['sarcasm'].apply(get_labels)
df.head()

,utterance,speaker,context,context_speakers,show,sarcasm
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,1
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,1
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,0
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,0
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,1


In [8]:
column_dict = {'utterance':0,'speaker':1,'context':2,'context_speaker':3,'show':4,'sarcasm':5}

In [9]:
data = df.to_numpy()
print(data[0][2][0])
print(data.shape)
print(len(data[:,[0]]))

I never would have identified the fingerprints of string theory in the aftermath of the Big Bang.
(690, 6)
690


In [10]:
#creating total vocab for the data
vocab = set()

for samp in data[:,0]:
  # print(samp)
  # break
  sent_token = sent_tokenize(samp)
  for j in sent_token:
    tokens = word_tokenize(j)
    for token in tokens:
      vocab.add(token)

print(len(vocab))

for samp in data[:,2]:
  for sent in samp:
    sent_token = sent_tokenize(sent)
    for j in sent_token:
      tokens = word_tokenize(j)
      for token in tokens:
        vocab.add(token)
print(len(vocab))


2194
4549


In [11]:
glove_dict = {}
with open("glove.840B.300d.txt", 'r', encoding="utf-8") as f:
  i=0
  for line in f:
    i+=1
    values = line.split()
    word = values[0]
    if(word in vocab):
      st = 1
      if(len(values)>301):
        st+= len(values) - 301
      vector = np.asarray(values[st:],'float32')
      glove_dict[word] = vector

print(len(glove_dict))

4428


In [12]:
for word in vocab:
  if(word not in glove_dict):
    temp_arr = [0.0]*300
    glove_dict[word] = np.asarray(temp_arr,'float32')

print(len(vocab))
print(len(glove_dict))

4549
4549


In [13]:
!pip install texttable


In [14]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-sparse==latest+{CUDA}      -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-cluster==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-spline-conv==latest+{CUDA} -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 1.6MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 24.3MB 193kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 21.5MB 251kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 6.4MB 366kB/s 
     |████████████████████████████████| 184kB 4.3MB/s 
     |████████████████████████████████| 235kB 5.7MB/s 
     |████████████████████████████████| 2.2MB 5.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.2-cp36-none-any.whl size=318842 sha256=b052a9cc34487f13ce6429229e0cb49a3a043cf1b916cb939cb51709dbfa1df5
  Stored in directory: /root/.cache/pip/wheels/ae/09/bb/09bf5950d6404198b0c61ea702536b126b50caf9f6040157d3
Succe

In [15]:
# import torch

class AttentionModule(torch.nn.Module):
    
    def __init__(self, args):
        
        super(AttentionModule, self).__init__()
        self.args = args
        self.filters_3 = 300
        self.filters_4 = 64
        self.setup_weights()
        self.init_parameters()

    def setup_weights(self):
        
        self.weight_matrix = torch.nn.Parameter(torch.Tensor(self.filters_3,
                                                             self.filters_3))

    def init_parameters(self):
       
        torch.nn.init.xavier_uniform_(self.weight_matrix)

    def forward(self, embedding):
        
        global_context = torch.mean(torch.matmul(embedding, self.weight_matrix), dim=0)
        transformed_global = torch.tanh(global_context)
        # print(type(transformed_global))
        # print(transformed_global.shape)
        sigmoid_scores = torch.sigmoid(torch.mm(embedding, transformed_global.view(-1, 1)))
        representation = torch.mm(torch.t(embedding), sigmoid_scores)
        return representation



In [117]:
class Model(torch.nn.Module):
    def __init__(self, args):
        
        super(Model, self).__init__()
        self.args = args
        self.tensor_neurons = 16
        self.bins = 16
        self.histogram = False
        self.setup_layers()

    def setup_layers(self):
        
        
        self.attention = AttentionModule(self.args)
      
        if(args=='ut'):
          self.fully_connected_first = torch.nn.Linear(300,128)
        
        if(args=='ut-sp'):
          self.fully_connected_first = torch.nn.Linear(321,128)

        if(args=='ut-ct'):
          self.fully_connected_first = torch.nn.Linear(300,128)

        if(args=='ut-ct-ext'):
          self.fully_connected_first = torch.nn.Linear(350,128)

        if(args=='ut-sh'):
          self.fully_connected_first = torch.nn.Linear(304,128)

        if(args=='ut-sp-ct'):
          self.fully_connected_first = torch.nn.Linear(321,128)

        if(args=='ut-sp-ct-cp-sh'):
          self.fully_connected_first = torch.nn.Linear(349,128)

       

        self.fully_connected_second = torch.nn.Linear(128,64)
        self.fully_connected_third = torch.nn.Linear(64,32)
        
        self.fully_connected_fourth = torch.nn.Linear(32,16)
        # self.fully_connected_fifth = torch.nn.Linear(16,8)
        self.scoring_layer = torch.nn.Linear(16,1)

  

    def forward(self, data):
        
        utter_feature = torch.from_numpy(np.array(data[0]))
        context_feature = []
        speaker_feature = []
        show_feature = []
        context_speaker_feature = []
        scores = []


        utter_atten = self.attention(utter_feature)

        if(args=='ut'):
          scores = utter_atten
          scores = torch.t(scores)

        if(args=='ut-sp'):
          speaker_feature = torch.from_numpy(np.array(data[1])).view(-1,1)
          scores = torch.cat((utter_atten,speaker_feature),dim=0).view(1,-1)
        
        if(args=='ut-ct'):
          context_feature = torch.from_numpy(np.array(data[1]))
          context_atten = self.attention(context_feature)
          scores = utter_atten.add(context_atten)
          scores = torch.t(scores)
          
        if(args=='ut-ct-ext'):
          context_feature = torch.from_numpy(np.array(data[1]))
          context_atten = self.attention(context_feature)
          scores = torch.cat((utter_atten,context_atten[:50]),dim = 0).view(-1,1)
          scores = torch.t(scores)
          # print(scores.shape)

        if(args=='ut-sh'):
          show_feature = torch.from_numpy(np.array(data[1])).view(-1,1)
          scores = torch.cat((utter_atten,show_feature),dim = 0).view(-1,1)
          scores = torch.t(scores)

        if(args=='ut-sp-ct'):
          speaker_feature = torch.from_numpy(np.array(data[1])).view(-1,1)
          context_feature = torch.from_numpy(np.array(data[2]))
          context_atten = self.attention(context_feature)
          scores = utter_atten.add(context_atten)
          # scores = torch.cat((utter_atten,context_atten),dim = 0).view(-1,1)
          scores = torch.cat((scores,speaker_feature),dim=0).view(1,-1)

        if(args=='ut-sp-ct-cp-sh'):
          speaker_feature = torch.from_numpy(np.array(data[1])).view(-1,1)
          context_feature = torch.from_numpy(np.array(data[2]))
          context_atten = self.attention(context_feature)
          context_speaker_feature = torch.from_numpy(np.array(data[3])).view(-1,1)
          show_feature = torch.from_numpy(np.array(data[4])).view(-1,1)
          scores = utter_atten.add(context_atten)
          # scores = torch.cat((utter_atten,context_atten),dim = 0).view(-1,1)
          scores = torch.cat((scores,speaker_feature),dim=0)
          # print(scores.shape)
          scores = torch.cat((scores,context_speaker_feature),dim=0)
          scores = torch.cat((scores,show_feature),dim=0).view(1,-1)
          # print(scores.shape)

        score1 = torch.nn.functional.relu(self.fully_connected_first(scores))
        score2 = torch.nn.functional.relu(self.fully_connected_second(score1))
        score3 = torch.nn.functional.relu(self.fully_connected_third(score2))
        score4 = torch.nn.functional.relu(self.fully_connected_fourth(score3))
        # score5 = torch.nn.functional.relu(self.fully_connected_fifth(score4))
        score = torch.sigmoid(self.scoring_layer(score4))
        
        return score,score4


In [118]:
class Trainer(object):
    
    def __init__(self, args,train_data,test_data,glove_dict):
        
        self.args = args
        self.train_data = train_data
        self.test_data = test_data
        self.glove_dict = glove_dict
        self.embedding_gen()
        self.setup_model()
 
    def setup_model(self):
        
        self.model = Model(self.args)
 
    def embedding_gen(self):

      speaker_cnt = 0
      context_speaker_cnt  = 0
      show_cnt = 0
      
      speaker_dict = {}
      context_speaker_dict = {}
      show_dict = {}
      
      for sample in self.train_data[:,[1,3,4]]:
        speaker = sample[0]
        context_speakers = sample[1]
        show = sample[2]
        if(speaker not in speaker_dict):
          speaker_dict[speaker] = speaker_cnt
          speaker_cnt+=1
        
        for context_speaker in context_speakers:
          if(context_speaker not in context_speaker_dict):
            context_speaker_dict[context_speaker] = context_speaker_cnt
            context_speaker_cnt+=1
        
        if(show not in show_dict):
          show_dict[show] = show_cnt
          show_cnt+=1

      for sample in self.test_data[:,[1,3,4]]:
        speaker = sample[0]
        context_speakers = sample[1]
        show = sample[2]
        if(speaker not in speaker_dict):
          speaker_dict[speaker] = speaker_cnt
          speaker_cnt+=1
        
        for context_speaker in context_speakers:
          if(context_speaker not in context_speaker_dict):
            context_speaker_dict[context_speaker] = context_speaker_cnt
            context_speaker_cnt+=1
        
        if(show not in show_dict):
          show_dict[show] = show_cnt
          show_cnt+=1
      print(speaker_dict)
      print(context_speaker_dict)
      print(show_dict)
      dim_speaker = len(speaker_dict)
      dim_context_speaker = len(context_speaker_dict)
      dim_show = len(show_dict)

      print('speaker_cnt: ',dim_speaker)
      print('context_speaker_cnt: ',dim_context_speaker)
      print('show_cnt: ',dim_show)
      
       
      print("\nGenerating for train\n")

      label_train_data = []
      for sample in self.train_data[:,5]:
        label = sample
        label_train_data.append(label)
      
      label_train_data = np.array(label_train_data)
      # label_train_data.resize(len(self.train_data),1)

      utter_train_data = []
      for sample in self.train_data[:,0]:
        feature_vec = []
        ut_text = sample
        sent_token = sent_tokenize(ut_text)
        for sent in sent_token:
          tokens = word_tokenize(sent)
          for token in tokens:
            feature_vec.append(self.glove_dict[token])
        
        utter_train_data.append(np.array(feature_vec))

      utter_train_data = np.array(utter_train_data)

      context_train_data = []
      for sample in self.train_data[:,2]:
        feature_vec = []
        for ct_sent in sample:
          ct_text = ct_sent
          sent_token = sent_tokenize(ct_text)
          for sent in sent_token:
            tokens = word_tokenize(sent)
            for token in tokens:
              feature_vec.append(self.glove_dict[token])
        context_train_data.append(np.array(feature_vec))
      
      context_train_data = np.array(context_train_data)

      speaker_train_data = []
      for sample in self.train_data[:,1]:
        feature_vec = [0]*dim_speaker
        # print(feature_vec)
        speaker = sample
        feature_vec[speaker_dict[speaker]] = 1
        # print(feature_vec)
        # return
        speaker_train_data.append(np.array(feature_vec))

      speaker_train_data = np.array(speaker_train_data)

      context_speaker_train_data = []
      for sample in self.train_data[:,3]:
        feature_vec = [0]*dim_context_speaker
        context_speaker = sample
        for spk in context_speaker:
          feature_vec[context_speaker_dict[spk]] = 1
        
        context_speaker_train_data.append(np.array(feature_vec))
      
      context_speaker_train_data = np.array(context_speaker_train_data)

      show_train_data = []
      for sample in self.train_data[:,4]:
        feature_vec = [0]*dim_show
        show = sample
        feature_vec[show_dict[show]] = 1
        show_train_data.append(np.array(feature_vec))

      show_train_data = np.array(show_train_data)
      

      print("\nGenerating for test\n")

      label_test_data = []
      for sample in self.test_data[:,5]:
        label = sample
        label_test_data.append(label)

      label_test_data = np.array(label_test_data)

      utter_test_data = []
      for sample in self.test_data[:,0]:
        feature_vec = []
        ut_text = sample
        sent_token = sent_tokenize(ut_text)
        for sent in sent_token:
          tokens = word_tokenize(sent)
          for token in tokens:
            feature_vec.append(self.glove_dict[token])
        utter_test_data.append(np.array(feature_vec))

      utter_test_data = np.array(utter_test_data)

      context_test_data = []
      for sample in self.test_data[:,2]:
        feature_vec = []
        for ct_sent in sample:
          ct_text = ct_sent
          sent_token = sent_tokenize(ct_text)
          for sent in sent_token:
            tokens = word_tokenize(sent)
            for token in tokens:
              feature_vec.append(self.glove_dict[token])
        context_test_data.append(np.array(feature_vec))
      
      context_test_data = np.array(context_test_data)

      speaker_test_data = []
      for sample in self.test_data[:,1]:
        feature_vec = [0]*dim_speaker
        speaker = sample
        feature_vec[speaker_dict[speaker]] = 1
        speaker_test_data.append(np.array(feature_vec))

      
      speaker_test_data = np.array(speaker_test_data)

      context_speaker_test_data = []
      for sample in self.test_data[:,3]:
        feature_vec = [0]*dim_context_speaker
        context_speaker = sample
        for spk in context_speaker:
          feature_vec[context_speaker_dict[spk]] = 1
        
        context_speaker_test_data.append(np.array(feature_vec))
      
      context_speaker_test_data = np.array(context_speaker_test_data)

      show_test_data = []
      for sample in self.test_data[:,4]:
        feature_vec = [0]*dim_show
        show = sample
        feature_vec[show_dict[show]] = 1
        show_test_data.append(np.array(feature_vec))

      show_test_data = np.array(show_test_data)

      self.train_dataset = []
      self.test_dataset = []
      
      if(args == 'ut'):  
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],label_test_data[i]])
        
      elif(args == 'ut-sp'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],speaker_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],speaker_test_data[i],label_test_data[i]])
      
      elif(args == 'ut-ct' or args=='ut-ct-ext'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],context_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],context_test_data[i],label_test_data[i]])
        
      elif(args == 'ut-sh'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],show_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],show_test_data[i],label_test_data[i]])

      elif(args=='ut-sp-ct'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],speaker_train_data[i],context_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],speaker_test_data[i],context_test_data[i],label_test_data[i]])


      elif(args=='ut-sp-cp'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],speaker_train_data[i],context_speaker_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],speaker_test_data[i],context_speaker_test_data[i],label_test_data[i]])

      elif(args=='ut-sp-sh'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],speaker_train_data[i],show_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],speaker_test_data[i],show_test_data[i],label_test_data[i]])

      elif(args == 'ut-sp-ct-cp-sh'):
        for i in range(len(self.train_data)):
          self.train_dataset.append([utter_train_data[i],speaker_train_data[i],context_train_data[i],context_speaker_train_data[i],show_train_data[i],label_train_data[i]])
        for i in range(len(self.test_data)):
          self.test_dataset.append([utter_test_data[i],speaker_test_data[i],context_test_data[i],context_speaker_test_data[i],show_test_data[i],label_test_data[i]])

      self.train_dataset = np.array(self.train_dataset)
      self.test_dataset = np.array(self.test_dataset)
      print(self.train_dataset.shape)
      print(self.test_dataset.shape)
 
            
    def create_batches(self):
       
        
        mod_batches = []
        np.random.shuffle(self.train_dataset)
        
        ind = 0
        while(ind<len(self.train_dataset)):  
          batches = []
          for num in range(1):
            batches.append(self.train_dataset[ind])
            ind += 1
           
          mod_batches.append(batches)
        return mod_batches
 
    def process_batch(self, batch):
        
        self.optimizer.zero_grad()
        losses = 0
        # print('process_batch')
        # it = 0
        for sample in batch:
            data = sample[:len(sample)-1]
            label = sample[-1]
            result = torch.from_numpy(np.array(label).reshape(1,1)).view(-1).float() 
            prediction,_ = self.model(data)
            losses = losses + torch.nn.functional.mse_loss(result, prediction)
        losses.backward(retain_graph=True)
        self.optimizer.step()
        loss = losses.item()
        return loss
 
    def fit(self):
      
 
        print("\nModel training.\n")
        self.optimizer = torch.optim.Adam(self.model.parameters(),
                                          lr=0.001,
                                          weight_decay=5*10**-4)
 
        self.model.train()
        
        epochs = 150
        min_loss = 10000
        min_loss_epoch = 0
        self.min_loss_model = self.model
        for epoch in range(epochs):
 
            batches = self.create_batches()
            
            self.loss_sum = 0
            main_index = 0
            for 
                loss_score = self.process_batch(batch)
                
                main_index = main_index + len(batch)
                # print(main_index)
                self.loss_sum = self.loss_sum + loss_score * len(batch)
                # self.loss_sum = self.loss_sum + loss_score
                loss = self.loss_sum/main_index
                
                
            # print(loss)
            if loss < min_loss:
                    min_loss = loss
                    min_loss_epoch = epoch
                    self.min_loss_model = self.model
                    print('min_loss: ',min_loss)
                    print(min_loss_epoch)
                    # print(self.min_loss_model)
            
 
        svm_feature = []
        # svm_target = []
        
        for sample in self.train_dataset:
          data = sample[:len(sample)-1]
            
          _,feature = self.min_loss_model(data)
          svm_feature.append(feature.tolist()[0])
          # svm_target.append(result)
 
        svm_target = []
        for sample in self.train_dataset:
          # data = graph_pair[:2]
          result = sample[-1]
          # _,feature = self.min_loss_model(data)
          svm_target.append(result)
 
 
        cnt0 = 0
        cnt1 = 0
        for i in svm_target:
          if(i==0):
            cnt0+=1
          else:
            cnt1+=1
        print(cnt0)
        print(cnt1)
 
        
        X = np.array(svm_feature)
        y = np.array(svm_target)
        print(y.shape)
        print(np.unique(y))
        from sklearn import svm
        from sklearn.metrics import confusion_matrix,accuracy_score
        self.svm_rbf_model = svm.SVC(kernel = 'rbf',gamma = 'scale', decision_function_shape = 'ovr')
        self.svm_rbf_model.fit(X,y)
        print("svm_model_trained")
        #now adding SVM rbf classifier for removing step function
        
    
 
    
    def score(self):
        
        print("\n\nModel evaluation.\n")
        
 
        model1 = self.min_loss_model
        model1.eval()
        self.scores = []
 
        predictions = []
        true_label = []
        for sample in self.test_dataset:
            
            data = sample[:len(sample)-1]
            true_label.append(sample[-1])
            # result = torch.from_numpy(np.array(label).reshape(1,2)).view(-1).float() 
            
            # print (type(graph_pair[2]))
 
            _,prediction = model1(data)
            predict = self.svm_rbf_model.predict(np.array(prediction.tolist())) 
            predictions.append(predict)
        
        return true_label,predictions,model1,self.svm_rbf_model
 
       

In [119]:
# print(type(data))
args = 'ut-sp-ct-cp-sh'
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from tqdm import tqdm, trange
import pickle
from sklearn.metrics import precision_recall_fscore_support, classification_report
skf = StratifiedKFold(n_splits=5, shuffle=True)
# kf = KFold(n_splits=5)
# kf.get_n_splits(data)
results = []
accur = []
print(type(df))
max_f1 = 0
for train_index, test_index in skf.split(data,df['sarcasm'].to_numpy()):
# for train_index, test_index in kf.split(data):  
  train_data, test_data = data[train_index], data[test_index]
  print(train_data.shape)
  print(test_data.shape)
  
  trainer = Trainer(args,train_data,test_data,glove_dict)
  trainer.fit()
  y_test,y_pred,atten_model,svm_model = (trainer.score())
  # filename_atten = args+'_atten_model_50.sav'
  # filename_svm = args+'_svm_model_50.sav'
  # filename_atten = args+'_atten_model_75.sav'
  # filename_svm = args+'_svm_model_75.sav'
  # filename_atten = args+'_atten_model_100.sav'
  # filename_svm = args+'_svm_model_100.sav'

  # filename_atten = args+'_atten_model_150.sav'
  # filename_svm = args+'_svm_model_150.sav'
  accur.append(accuracy_score(y_test, y_pred))
  results.append(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

  # if(max_f1 < results[-1][2]):
  #   max_f1 = results[-1][2]
  #   pickle.dump(atten_model, open(filename_atten, 'wb'))
  #   pickle.dump(svm_model,open(filename_svm,'wb'))
  
  print(accur[-1])
  print(classification_report(y_test, y_pred))
  

avg = [0,0,0,0]
for i in range(5):
    x, y, z, _ = results[i]
    avg[0] += x
    avg[1] += y
    avg[2] += z
    avg[3] += accur[i]
avg[0]/=5
avg[1]/=5
avg[2]/=5
avg[3]/=5


print(f"Avg Accuracy: {avg[3]:.3f}")
print(f"Avg weighted precision: {avg[0]:.3f} :: Avg weighted recall: {avg[1]:.3f} :: Avg weighted F1: {avg[2]:.3f}")





<class 'pandas.core.frame.DataFrame'>
(552, 6)
(138, 6)
{'SHELDON': 0, 'HOWARD': 1, 'LEONARD': 2, 'RAJ': 3, 'PERSON': 4, 'PENNY': 5, 'AMY': 6, 'PERSON3': 7, 'BERNADETTE': 8, 'PERSON1': 9, 'CHANDLER': 10, 'ROSS': 11, 'MONICA': 12, 'JOEY': 13, 'RACHEL': 14, 'PHOEBE': 15, 'DOROTHY': 16, 'ROSE': 17, 'MEMBER-GIRL': 18, 'MODERATOR': 19, 'MEMBER-BOY': 20}
{'LEONARD': 0, 'SHELDON': 1, 'HOWARD': 2, 'PENNY': 3, 'PERSON': 4, 'RAJ': 5, 'PERSON1': 6, 'PERSON2': 7, 'BERNADETTE': 8, 'AMY': 9, 'JOEY': 10, 'CHANDLER': 11, 'ROSS': 12, 'MONICA': 13, 'RACHEL': 14, 'PHOEBE': 15, 'DOROTHY': 16, 'ROSE': 17, 'BLANCHE': 18, 'SOPHIA': 19, 'MODERATOR': 20, 'SCOTT': 21, 'MEMBER-BOY': 22, 'MEMBER-GIRL': 23}
{'BBT': 0, 'FRIENDS': 1, 'GOLDENGIRLS': 2, 'SARCASMOHOLICS': 3}
speaker_cnt:  21
context_speaker_cnt:  24
show_cnt:  4

Generating for train


Generating for test

(552, 6)
(138, 6)

Model training.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:302: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


min_loss:  0.2517807266904392
0
min_loss:  0.25049828138013464
1
min_loss:  0.2459788139609188
2
min_loss:  0.23910840533757885
3
min_loss:  0.2257591367440224
4
min_loss:  0.19994825129491764
5
min_loss:  0.16552663568770623
6
min_loss:  0.13452120971126472
7


KeyboardInterrupt: ignored